# PE Halo

This notebook will exercise your learning of Serialbox and porting from Fortran to Python.



In [24]:
%%writefile pe_halo.F90

subroutine compute_pe_halo(is, ie, js, je, isd, ied, jsd, jed, npz, ptop, pe, delp)
    integer, intent(in) :: is, ie, js, je, isd, ied, jsd, jed, npz
    real, intent(in) :: ptop
    real, intent(in), dimension(isd:ied,jsd:jed,npz):: delp
    real, intent(inout), dimension(is-1:ie+1,npz+1,js-1:je+1):: pe
    integer:: i,j,k

    do j=js,je
     pe(is-1,1,j) = ptop
     pe(ie+1,1,j) = ptop
     do k=1,npz
        pe(is-1,k+1,j) = pe(is-1,k,j) + delp(is-1,j,k)
        pe(ie+1,k+1,j) = pe(ie+1,k,j) + delp(ie+1,j,k)
     enddo
    enddo
    
    do i=is-1,ie+1
     pe(i,1,js-1) = ptop
     pe(i,1,je+1) = ptop
     do k=1,npz
        pe(i,k+1,js-1) = pe(i,k,js-1) + delp(i,js-1,k)
        pe(i,k+1,je+1) = pe(i,k,je+1) + delp(i,je+1,k)
     enddo
  enddo
end subroutine

program pe_halo

  implicit none
    
  integer :: is, ie, js, je, isd, ied, jsd, jed, npz
  real :: ptop
  real, allocatable, dimension(:,:,:) :: delp, pe
  !$ser init directory='./data_pe' prefix='port' unique_id=.true.
  !$ser mode write
  !$ser on
  is = 1
  ie = 48
  js = 1
  je = 48
  isd = -2
  ied = 51
  jsd = -2
  jed = 51
  npz = 79
  ptop = 300.0
  allocate(delp(isd:ied, jsd:jed, npz))
  allocate(pe(is-1:ie+1,npz+1,js-1:je+1))
  !$ser savepoint 'pe_halo-in'
  !$ser data ptop=ptop pe=pe delp=delp is=is ie=ie js=js je=je isd=isd ied=ied jsd=jsd jed=jed npz=npz
  call compute_pe_halo(is, ie, js, je, isd, ied, jsd, jed, npz, ptop, pe, delp)
  !$ser savepoint 'pe_halo-out'
  !$ser data pe=pe
  !$ser cleanup
end program

Overwriting pe_halo.F90


In [25]:
%%bash
python3 ${SERIALBOX_ROOT}/python/pp_ser/pp_ser.py -s -v --output=s_pe_halo.F90 pe_halo.F90
gfortran -O3 -cpp -DSERIALIZE \
    -o serialize_pe_halo s_pe_halo.F90 \
    -I${SERIALBOX_ROOT}/include \
    ${SERIALBOX_ROOT}/lib/libSerialboxFortran.a \
    ${SERIALBOX_ROOT}/lib/libSerialboxC.a \
    ${SERIALBOX_ROOT}/lib/libSerialboxCore.a \
    -lpthread -lstdc++ -lstdc++fs
rm -rf ./data_pe
./serialize_pe_halo
ls -lh data_pe

Processing file pe_halo.F90
 >>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<
 >>> WARNING: SERIALIZATION IS ON <<<
 >>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<
total 2.5M
-rw-r--r-- 1 rgeorge noaa-hpc 1.7K Nov  9 15:28 ArchiveMetaData-port.json
-rw-r--r-- 1 rgeorge noaa-hpc  14K Nov  9 15:28 MetaData-port.json
-rw-r--r-- 1 rgeorge noaa-hpc 900K Nov  9 15:28 port_delp.dat
-rw-r--r-- 1 rgeorge noaa-hpc    4 Nov  9 15:28 port_ie.dat
-rw-r--r-- 1 rgeorge noaa-hpc    4 Nov  9 15:28 port_ied.dat
-rw-r--r-- 1 rgeorge noaa-hpc    4 Nov  9 15:28 port_is.dat
-rw-r--r-- 1 rgeorge noaa-hpc    4 Nov  9 15:28 port_isd.dat
-rw-r--r-- 1 rgeorge noaa-hpc    4 Nov  9 15:28 port_je.dat
-rw-r--r-- 1 rgeorge noaa-hpc    4 Nov  9 15:28 port_jed.dat
-rw-r--r-- 1 rgeorge noaa-hpc    4 Nov  9 15:28 port_js.dat
-rw-r--r-- 1 rgeorge noaa-hpc    4 Nov  9 15:28 port_jsd.dat
-rw-r--r-- 1 rgeorge noaa-hpc    4 Nov  9 15:28 port_npz.dat
-rw-r--r-- 1 rgeorge noaa-hpc 1.6M Nov  9 15:28 port_pe.dat
-rw-r--r-- 1 rgeorge noaa-hpc    4 Nov 

### Python

In [36]:
#!/usr/bin/env python3

import numpy as np
import gt4py.gtscript as gtscript
import gt4py.storage as gt_storage
import sys
import os
sys.path.append(os.environ.get('SERIALBOX_ROOT')+ '/python')
import serialbox as ser

field = gtscript.Field[np.float64]
backend="numpy"
origin=(0, 0, 0)
@gtscript.stencil(backend=backend)
def edge_pe(pe: field, delp: field, ptop: float):
    with computation(FORWARD):
        with interval(0, 1):
            pe[0, 0, 0] = ptop
        with interval(1, None):
            pe[0, 0, 0] = pe[0, 0, -1] + delp[0, 0, -1]

def compute(pe, delp, ptop, is_, ie_, js_, je_, npz):
    edge_domain_x = (1, je_ - js_ + 1, npz + 1)
    edge_pe(pe, delp, ptop, origin=(0, 1, 0), domain=edge_domain_x)
    edge_pe(pe, delp, ptop, origin=(ie_ + 1, 1, 0), domain=edge_domain_x)
    edge_domain_y = (ie_ - is_ + 3, 1, npz + 1)
    edge_pe(pe, delp, ptop, origin=(0, 0, 0), domain=edge_domain_y)
    edge_pe(pe, delp, ptop, origin=(0, je + 1, 0), domain=edge_domain_y)

serializer = ser.Serializer(ser.OpenModeKind.Read,'./data_pe', 'port')

sp_in = serializer.get_savepoint('pe_halo-in')

pe = serializer.read('pe',  sp_in[0])
delp = serializer.read('delp',  sp_in[0])
ptop = serializer.read('ptop',  sp_in[0])
fortran2python_offset = 2
is_ = int(serializer.read('is',  sp_in[0])) + fortran2python_offset
ie_ = int(serializer.read('ie',  sp_in[0])) + fortran2python_offset
js_ = int(serializer.read('js',  sp_in[0])) + fortran2python_offset
je_ = int(serializer.read('je',  sp_in[0])) + fortran2python_offset
isd = int(serializer.read('isd',  sp_in[0])) + fortran2python_offset
ied = int(serializer.read('ied',  sp_in[0])) + fortran2python_offset
jsd = int(serializer.read('jsd',  sp_in[0])) + fortran2python_offset
jed = int(serializer.read('jed',  sp_in[0])) + fortran2python_offset
npz = int(serializer.read('npz',  sp_in[0])) 
pe = np.moveaxis(pe, 1, 2)
storage_shape=(ie_ + 1 - (is_ - 1) + 1, je_ + 1 - (js_ - 1) + 1, npz+1)
pe =  gt_storage.from_array(data=pe, backend=backend,  dtype=np.float64,
                            default_origin=origin, shape=storage_shape)
delp_overlap = np.zeros(storage_shape)
delp_overlap[:, :, 0:npz] = delp[is_ - 1: ie_ + 2, js_ - 1: je_ + 2,:]
delp = gt_storage.from_array(data=delp_overlap,  backend=backend,  dtype=np.float64,
                            default_origin=origin, shape=storage_shape,)
compute(pe, delp, ptop, is_, ie_, js_, je_, npz)
# Get regression data
sp_out = serializer.get_savepoint('pe_halo-out')
pe_ref = serializer.read('pe',  sp_out[0])
# reformat computed value back
pe =  np.moveaxis(pe, 2, 1)
# compare answers
try:
    assert np.allclose(pe_ref, pe), "pe does not match!"
except AssertionError as msg:
    print(msg)
    
print("Finished running comparison tests!")

TypeError: The type of parameter 'ptop' is '<class 'numpy.ndarray'>' instead of 'float64'